In [ ]:
import numpy as np
import pandas as pd
from keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.layers import Dropout
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

In [ ]:
dataset_train = pd.read_csv('/dados/yahoo_magalu_train.csv')

In [ ]:
training_set = dataset_train.iloc[:, 1:2].values

In [ ]:
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
X_train = []
Y_train = []
for i in range(60, 1245):
  X_train.append(training_set_scaled[i-60:i, 0])
  Y_train.append(training_set_scaled[i, 0])

In [ ]:
X_train, Y_train = np.array(X_train), np.array(Y_train)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))

In [ ]:
regressor = Sequential()

In [ ]:
regressor.add(LSTM(units = 50, return_sequences = True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))

In [ ]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
regressor.fit(X_train, Y_train, epochs = 100, batch_size = 32)

In [ ]:
dataset_test = pd.read_csv('<path>/yahoo_magalu_test.csv')

In [ ]:
real_stock_price = dataset_test.iloc[:, 1:2].values

In [ ]:
dataset_total = pd.concat((dataset_train['Open'],dataset_test['Open']), axis = 0)

In [ ]:
inputs = dataset_total[len(dataset_total) - len(dataset_test)-60:].values

In [ ]:
inputs = inputs.reshape(-1,1)

In [ ]:
inputs = sc.transform(inputs)

In [ ]:
X_test = []
for i in range(60, 79):
  X_test.append(inputs[i-60:i, 0])
  X_test = np.array(X_test)

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
predicted_stock_price = regressor.predict(X_test)

In [ ]:
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'Dados Reais de Açõesda Magazine Luiza')
plt.plot(predicted_stock_price, color = 'blue', label = 'Dados Previstosde Ações da Magazine Luiza')
plt.title('Previsão de Preços de Ações')
plt.xlabel('Tempo')
plt.ylabel('Preços de Ações da Magazine Luiza')
plt.legend()
plt.show()